In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

In [22]:
file_path = r"C:\Users\advait\Desktop\converted_distillation_data.csv"
df = pd.read_csv(file_path)
inputs = ['Feed_Flow_Rate', 'Feed_Composition', 'Reflux_Ratio', 'Boil_Up_Ratio']
X = df[inputs]

In [30]:
rf_model = joblib.load('rf_model.pkl')
xgb_model = joblib.load('xgb_model.pkl')

In [32]:
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [34]:
y_pred1 = rf_model.predict(X_test)
y_pred2 = xgb_model.predict(X_test)


In [36]:
y_pred_combined = (y_pred1 + y_pred2) / 2


In [38]:
inputs = [[350, 2.5, 100, 2000]]
input_df = pd.DataFrame(inputs, columns=['Feed_Flow_Rate','Feed_Composition','Reflux_Ratio','Boil_Up_Ratio'])
ideal_values = [99.6, 0.04, 1250, 1120]

In [42]:
def objective_function(inputs):
    input_df = pd.DataFrame([inputs], columns=['Feed_Flow_Rate', 'Feed_Composition', 'Reflux_Ratio', 'Boil_Up_Ratio'])
    y_pred1 = rf_model.predict(input_df)
    y_pred2 = xgb_model.predict(input_df)
    y_pred_combined = (y_pred1 + y_pred2) / 2
    error = np.mean((y_pred_combined - ideal_values) ** 2)
    return error

initial_inputs = [75, 5, 2.5, 0.1]
bounds = [(50, 100),
    (0, 50),
    (0.5, 10.0),
    (0.0,0.3)]
result = minimize(objective_function, initial_inputs, bounds=bounds, method='L-BFGS-B')
optimized_inputs = result.x
print("Optimized Inputs:", optimized_inputs)

Optimized Inputs: [75.   5.   2.5  0.1]
